In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_vertical_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,8114.60,-0.06,-0.00,0.05,0.00,-0.01,0.00,-0.00,-0.00
3,-0.06,2283.38,-0.02,0.01,-0.01,0.00,0.01,0.00,0.00
4,-0.00,-0.02,639.90,0.01,0.00,0.00,-0.00,-0.00,0.00
5,0.05,0.01,0.01,166.78,0.00,-0.00,0.00,0.00,0.00
6,0.00,-0.01,0.00,0.00,40.68,-0.00,-0.00,0.00,0.00
7,-0.01,0.00,0.00,-0.00,-0.00,9.46,0.00,0.00,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.13,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.49,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00003,0.00001,-0.00002,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,0.00007,0.00001,0.00001
4,-0.00000,-0.00001,1.00000,0.00004,0.00000,0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00000,-0.00003,0.00007,0.00001,0.00002
6,0.00000,-0.00003,0.00000,0.00000,1.00000,-0.00001,-0.00000,0.00003,0.00001
7,-0.00003,0.00002,0.00000,-0.00003,-0.00001,1.00000,0.00002,0.00000,0.00001
8,0.00001,0.00007,-0.00005,0.00007,-0.00000,0.00002,1.00000,0.00004,0.00001
9,-0.00002,0.00001,-0.00001,0.00001,0.00003,0.00000,0.00004,1.00000,-0.00001
10,-0.00000,0.00001,0.00002,0.00002,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.11436860094193584

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.32688396e+06 5.70570268e+05 4.38858915e+04 2.89890049e+03
 1.65709324e+02 8.40870098e+00 3.82592996e-01 1.58729022e-02
 1.90053769e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999223,-0.038087,-0.009791,-0.002529,-0.000618,-0.000145,-0.000032,-0.000008,-0.000002
1,0.037595,0.998278,-0.043725,-0.010431,-0.002535,-0.000587,-0.000130,-0.000031,-0.000007
2,0.011281,0.042763,0.997859,-0.046943,-0.010601,-0.002453,-0.000556,-0.000128,-0.000030
3,0.003402,0.012160,0.045779,0.997437,-0.052234,-0.011327,-0.002537,-0.000588,-0.000138
4,0.000994,0.003541,0.012656,0.050827,0.996618,-0.061780,-0.013038,-0.003003,-0.000707
5,0.000289,0.001022,0.003647,0.014035,0.059807,0.994786,-0.079381,-0.017187,-0.004030
6,0.000085,0.000300,0.001083,0.004138,0.017022,0.075654,0.989250,-0.121070,-0.026345
7,0.000028,0.000100,0.000357,0.001367,0.005607,0.024161,0.110026,0.965770,-0.233592
8,0.000014,0.000049,0.000176,0.000676,0.002774,0.011935,0.052916,0.228747,0.971969


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00077694933483341,
 0.0017218958395693873,
 0.0021414192103990093,
 0.0025626238911795607,
 0.0033823947658864206,
 0.00521419650468935,
 0.01074969062797737,
 0.03422977578001818,
 0.02803077063401216]